# Batch trials

A notebook for trialling the developed methods on extended numbers of files...

In [1]:
%matplotlib qt
import os
import sys
import time

import matplotlib.pyplot as plt
import skimage.feature
import skimage.filters
import skimage.morphology

import hyperspy.api as hs

sys.path.append("../")
import windows

In [2]:
DATA_DIRECTORY = "../data"
data_filenames = []

# Make a list of filenames
for dataset_directory in os.listdir(DATA_DIRECTORY):
    dataset_directory_path = os.path.join(DATA_DIRECTORY, dataset_directory)
    data_filenames.append({
        "dataset_name": dataset_directory,
        "dataset_file_list": [
            os.path.join(dataset_directory_path, filename) 
            for filename in os.listdir(dataset_directory_path) 
            if filename[-4:] == ".dm3" or filename[-4:] == ".emd"
        ]
    })

In [3]:
selected_radius = 20
inner_background_radius = 50
outer_background_radius = 75

In [4]:
peak_intensities_list = []

In [5]:
print("Processing", len(data_filenames[1]["dataset_file_list"]), "files")
for filename in data_filenames[1]["dataset_file_list"]:
    s = hs.load(filename)
    s_dilated = skimage.filters.gaussian(s.data, sigma=selected_radius/3)
    # mean_filter_footprint = skimage.morphology.disk(selected_radius//2)
    # s_dilated = skimage.filters.rank.mean(s.data, mean_filter_footprint)
#     s_dilated = s.data
    peaks_list = skimage.feature.peak_local_max(s.data[:], min_distance=100, threshold_abs=100, exclude_border=True, num_peaks=12)
    peak_intensities = []
    image_window = windows.CircularWindow(selected_radius, inner_background_radius, outer_background_radius, s_dilated)
    for peak_point in peaks_list:
        peak_intensities.append(image_window.get_integrated_intensity(*peak_point))
        
#     fig, ax = plt.subplots()
#     ax.imshow(s_dilated, vmin=0, vmax=250)
#     for peak_point in peaks_list:
#         circle = plt.Circle(peak_point[::-1], outer_background_radius*1.1, color='g', fill=False)
#         ax.add_patch(circle)
#     plt.show()
    
    peak_intensities_list.append(peak_intensities)
    print("Processed another")

Processing 13 files
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another
Processed another


### Now normalise by the brightest peak in each image

In [6]:
norm_pk_int_lst = []
for peak_intensity_list in peak_intensities_list:
    max_peak_val = max(peak_intensity_list)
    normalised_list = [intensity / max_peak_val for intensity in peak_intensity_list]
    norm_pk_int_lst.append(normalised_list)

In [7]:
flat_norm_pk_int_lst = [intensity for peak_intensity_list in norm_pk_int_lst for intensity in peak_intensity_list]

In [8]:
fig, ax = plt.subplots()
ax.hist(flat_norm_pk_int_lst, bins=100)
plt.show()